In [1]:
import os
from time import time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from huggingface_hub import login
from sklearn.model_selection import StratifiedKFold, KFold
tqdm.pandas()

# Change the working directory to the directory containing the script
os.chdir("/group-volume/binfeng/wsdm/stage_qft")
from utils import *


/home/user/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = "microsoft/phi-4"
MAX_LENGTH = 4000
MAX_PROMPT_LENGTH = 800

## Tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.save_pretrained("/group-volume/binfeng/wsdm/tokenizer/phi4")

('/group-volume/binfeng/wsdm/tokenizer/phi4/tokenizer_config.json',
 '/group-volume/binfeng/wsdm/tokenizer/phi4/special_tokens_map.json',
 '/group-volume/binfeng/wsdm/tokenizer/phi4/vocab.json',
 '/group-volume/binfeng/wsdm/tokenizer/phi4/merges.txt',
 '/group-volume/binfeng/wsdm/tokenizer/phi4/added_tokens.json',
 '/group-volume/binfeng/wsdm/tokenizer/phi4/tokenizer.json')

## Prepare Data

In [4]:
ft = pd.read_parquet("/group-volume/binfeng/wsdm/stage_qft/data/ft48k_calibrated.parquet")
ft.dropna(inplace=True)
ft["text"] = ft.apply(lambda x: format_text(tokenizer, x.prompt, x.response_a, x.response_b, 
                                                max_len=MAX_LENGTH, max_prompt_len=MAX_PROMPT_LENGTH, reverse=True), axis=1)
ft["label"] = ft.apply(lambda x: format_label(x.winner, reverse=True), axis=1)


Token indices sequence length is longer than the specified maximum sequence length for this model (18891 > 16384). Running this sequence through the model will result in indexing errors


In [5]:
def switch_logits(x):
    return [x[1], x[0]]
ft["logits_qwencd_cali"] = ft["logits_qwencd_cali"].apply(switch_logits)
ft["logits_qwen32_cali"] = ft["logits_qwen32_cali"].apply(switch_logits)
ft["logits_qwen14_cali"] = ft["logits_qwen14_cali"].apply(switch_logits)

In [6]:
ft.head()

,prompt,response_a,response_b,winner,language,text,text_reverse,input_ids,input_ids_reverse,fold,logits_model_a,logits_model_b,logits_qwen32_cali,logits_qwencd_cali,logits_qwen14_cali,label
0,draw a unicorn using asciii,Sure! Here's a simple ASCII art representation...,Sure! Here’s a simple ASCII art representation...,model_a,English,<|User Prompt|>\ndraw a unicorn using asciii\n...,<|User Prompt|>\ndraw a unicorn using asciii\n...,"[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...","[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...",0,-0.298828,-2.351562,"[-2.3515625, -0.298828125]","[-0.875, 0.849609375]","[-2.96875, 0.250244140625]",1
1,"Сделать декомпозицию процесса ""выполнить ремон...","## Декомпозиция процесса ""Выполнить ремонт"" (D...","## Декомпозиция процесса ""Выполнить ремонт"" (д...",model_b,Russian,<|User Prompt|>\nСделать декомпозицию процесса...,<|User Prompt|>\nСделать декомпозицию процесса...,"[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...","[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...",1,-2.757812,0.110840,"[0.11083984375, -2.7578125]","[1.296875, -1.34765625]","[0.2568359375, -3.0]",0
2,NOMBRE DE LA ACTIVIDAD: Análisis descriptivo t...,"En el proceso de liofilización de alimentos, p...",## Análisis termodinámico de la liofilización:...,model_a,Spanish,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,"[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...","[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...",2,-0.080078,-2.523438,"[-2.5234375, -0.080078125]","[-1.16796875, 1.13671875]","[-2.625, -0.0933837890625]",1
3,find the logic error in this code and explain ...,The logic error in the provided code is relate...,The code is well-structured and clean. However...,model_a,English,<|User Prompt|>\nfind the logic error in this ...,<|User Prompt|>\nfind the logic error in this ...,"[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...","[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...",3,-0.859375,-1.820312,"[-1.8203125, -0.859375]","[-0.3662109375, 0.36328125]","[-3.09375, 0.34912109375]",1
4,"\nДобрый день, коллега. От Вас требуется завер...",```python\n \n \n \n \n \n \...,"В вашем коде есть несколько импортов, которые ...",model_b,Russian,"<|User Prompt|>\n\nДобрый день, коллега. От Ва...","<|User Prompt|>\n\nДобрый день, коллега. От Ва...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...",0,-1.765625,-0.835938,"[-0.8359375, -1.765625]","[1.09765625, -1.109375]","[-0.513671875, -2.125]",0


In [7]:
def tokenizer_func(example):
    return tokenizer(
        example["text"], 
        padding='max_length', 
        max_length=MAX_LENGTH,
        truncation=True,
        return_tensors='np'
    )

raw_dataset = DatasetDict({})

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=66)
for i, (train_index, val_index) in enumerate(skf.split(ft, ft["language"])):
    ft_train, ft_val = ft.iloc[train_index], ft.iloc[val_index]
    print(len(ft_train), len(ft_val))
    ft_train_dataset = Dataset.from_pandas(ft_train[["text", "label", "logits_qwencd_cali", "logits_qwen32_cali", "logits_qwen14_cali"]])
    ft_val_dataset = Dataset.from_pandas(ft_val[["text", "label", "logits_qwencd_cali", "logits_qwen32_cali", "logits_qwen14_cali"]])
    raw_dataset[f"ft_train_fold{i}"] = ft_train_dataset
    raw_dataset[f"ft_val_fold{i}"] = ft_val_dataset


tokenized_dataset = raw_dataset.map(tokenizer_func, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset = tokenized_dataset.remove_columns(['__index_level_0__', 'text'])
tokenized_dataset


/home/user/.local/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


38749 9688
38749 9688
38750 9687
38750 9687
38750 9687


Map: 100%|██████████| 9687/9687 [00:10<00:00, 909.60 examples/s] 


DatasetDict({
    ft_train_fold0: Dataset({
        features: ['labels', 'logits_qwencd_cali', 'logits_qwen32_cali', 'logits_qwen14_cali', 'input_ids', 'attention_mask'],
        num_rows: 38749
    })
    ft_val_fold0: Dataset({
        features: ['labels', 'logits_qwencd_cali', 'logits_qwen32_cali', 'logits_qwen14_cali', 'input_ids', 'attention_mask'],
        num_rows: 9688
    })
    ft_train_fold1: Dataset({
        features: ['labels', 'logits_qwencd_cali', 'logits_qwen32_cali', 'logits_qwen14_cali', 'input_ids', 'attention_mask'],
        num_rows: 38749
    })
    ft_val_fold1: Dataset({
        features: ['labels', 'logits_qwencd_cali', 'logits_qwen32_cali', 'logits_qwen14_cali', 'input_ids', 'attention_mask'],
        num_rows: 9688
    })
    ft_train_fold2: Dataset({
        features: ['labels', 'logits_qwencd_cali', 'logits_qwen32_cali', 'logits_qwen14_cali', 'input_ids', 'attention_mask'],
        num_rows: 38750
    })
    ft_val_fold2: Dataset({
        features: ['labe

In [10]:
i = 3
print(tokenizer.decode(tokenized_dataset["ft_train_fold0"][i]["input_ids"], skip_special_tokens=False))
print("**label:", tokenized_dataset["ft_train_fold0"][i]["labels"])

<|User Prompt|>

Добрый день, коллега. От Вас требуется завершение выполнения следующей разработки:

Нужен препроцессор Си (макропроцессор) значительно отличающийся от стандартного препроцессора Си. Целью разработки является упрощенная генерация Си кода и структур данных из метаданных. Для моих целей стандартный препроцессор Си недостаточно мощен, а макропроцессор M4 не интегрирован с Си синтаксисом. Язык реализации разработки - python, писать надо без написания новых классов (это жесткое неизменяемое требование, пишите функции, не классы. Будет хоть один класс - уволю!). Все описанное ниже надо реализовать в полном объеме.

Основные отличия от стандартного препроцессора Си:
*) символом начала директивы является символ @ а не # (во избежание пересечения со стандартным препроцессором). Кроме того лексически @ - не обязательно первый символ в строке, перед @ может быть некоторое количество white space c начала строки, но обязательно @ первый символ перед идентификатором директивы. Практи

In [11]:
tokenized_dataset.save_to_disk("/group-volume/binfeng/wsdm/stage_qft/dataset/tokenized_phi4_rev")

Saving the dataset (1/1 shards): 100%|██████████| 9687/9687 [00:00<00:00, 74737.64 examples/s]


In [14]:
ft_train

,prompt,response_a,response_b,winner,language,text,text_reverse,input_ids,input_ids_reverse,fold,logits_model_a,logits_model_b,logits_qwen32_cali,logits_qwencd_cali,logits_qwen14_cali,label
0,draw a unicorn using asciii,Sure! Here's a simple ASCII art representation...,Sure! Here’s a simple ASCII art representation...,model_a,English,<|User Prompt|>\ndraw a unicorn using asciii\n...,<|User Prompt|>\ndraw a unicorn using asciii\n...,"[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...","[27, 91, 1474, 59501, 91, 397, 7633, 264, 8183...",0,-0.298828,-2.351562,"[-2.3515625, -0.298828125]","[-0.875, 0.849609375]","[-2.96875, 0.250244140625]",1
1,"Сделать декомпозицию процесса ""выполнить ремон...","## Декомпозиция процесса ""Выполнить ремонт"" (D...","## Декомпозиция процесса ""Выполнить ремонт"" (д...",model_b,Russian,<|User Prompt|>\nСделать декомпозицию процесса...,<|User Prompt|>\nСделать декомпозицию процесса...,"[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...","[27, 91, 1474, 59501, 91, 397, 19311, 57930, 1...",1,-2.757812,0.110840,"[0.11083984375, -2.7578125]","[1.296875, -1.34765625]","[0.2568359375, -3.0]",0
2,NOMBRE DE LA ACTIVIDAD: Análisis descriptivo t...,"En el proceso de liofilización de alimentos, p...",## Análisis termodinámico de la liofilización:...,model_a,Spanish,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,<|User Prompt|>\nNOMBRE DE LA ACTIVIDAD: Análi...,"[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...","[27, 91, 1474, 59501, 91, 397, 45, 62011, 3385...",2,-0.080078,-2.523438,"[-2.5234375, -0.080078125]","[-1.16796875, 1.13671875]","[-2.625, -0.0933837890625]",1
3,find the logic error in this code and explain ...,The logic error in the provided code is relate...,The code is well-structured and clean. However...,model_a,English,<|User Prompt|>\nfind the logic error in this ...,<|User Prompt|>\nfind the logic error in this ...,"[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...","[27, 91, 1474, 59501, 91, 397, 3903, 279, 1221...",3,-0.859375,-1.820312,"[-1.8203125, -0.859375]","[-0.3662109375, 0.36328125]","[-3.09375, 0.34912109375]",1
4,"\nДобрый день, коллега. От Вас требуется завер...",```python\n \n \n \n \n \n \...,"В вашем коде есть несколько импортов, которые ...",model_b,Russian,"<|User Prompt|>\n\nДобрый день, коллега. От Ва...","<|User Prompt|>\n\nДобрый день, коллега. От Ва...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...","[27, 91, 1474, 59501, 91, 1339, 24110, 131014,...",0,-1.765625,-0.835938,"[-0.8359375, -1.765625]","[1.09765625, -1.109375]","[-0.513671875, -2.125]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48430,续写，形成对仗：感谢美诗，滋润我心，感谢佳句,感谢美诗，滋润我心；感谢佳句，启迪我思。,感谢美诗，滋润我心，感谢佳句，**点亮我眼。**,model_a,Chinese,<|User Prompt|>\n续写，形成对仗：感谢美诗，滋润我心，感谢佳句\n\n<|R...,<|User Prompt|>\n续写，形成对仗：感谢美诗，滋润我心，感谢佳句\n\n<|R...,"[27, 91, 1474, 59501, 91, 397, 99448, 61443, 3...","[27, 91, 1474, 59501, 91, 397, 99448, 61443, 3...",2,-0.164062,-2.484375,"[-2.484375, -0.1640625]","[-0.98046875, 0.984375]","[-0.4599609375, -2.25]",1
48431,扩写到150字：与英国帝国理工学院、美国明尼苏达州立大学、伍兹霍尔海洋研究所等知名高校与研究...,我们的机构积极拓展与国际顶尖学府和研究机构的合作网络，在自主潜水器的关键技术研究领域，与英国...,以下是扩写到150字的内容：\n\n与英国帝国理工学院、美国明尼苏达州立大学、伍兹霍尔海洋研...,model_a,Chinese,<|User Prompt|>\n扩写到150字：与英国帝国理工学院、美国明尼苏达州立大学、...,<|User Prompt|>\n扩写到150字：与英国帝国理工学院、美国明尼苏达州立大学、...,"[27, 91, 1474, 59501, 91, 397, 99834, 61443, 2...","[27, 91, 1474, 59501, 91, 397, 99834, 61443, 2...",3,-1.394531,-1.230469,"[-1.23046875, -1.39453125]","[0.053955078125, -0.030517578125]","[-0.650390625, -2.046875]",1
48432,原名赵亮的王雁南女士的爸爸，因为涉及了什么事件，被软禁起来？,王雁南女士的原名是赵亮，她的父亲是赵紫阳。赵紫阳是中国改革开放时期的重要领导人之一，曾担任国...,王雁南女士的父亲赵亮因涉及“反右派”事件而被软禁。反右派运动是中国在1957年进行的一次政治...,model_a,Chinese,<|User Prompt|>\n原名赵亮的王雁南女士的爸爸，因为涉及了什么事件，被软禁起来...,<|User Prompt|>\n原名赵亮的王雁南女士的爸爸，因为涉及了什么事件，被软禁起来...,"[27, 91, 1474, 59501, 91, 397, 52129, 13072, 1...","[27, 91, 1474, 59501, 91, 397, 52129, 13072, 1...",0,0.140442,-2.828125,"[-2.828125, 0.14044189453125]","[-0.7578125, 0.728515625]","[-1.55859375, -1.04296875]",1
48433,\n太興燒味會否結業